In [1]:
import numpy as np
# Modeling & Preprocessing
from keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dense, Dropout, LSTM, Input, TimeDistributed, Bidirectional
from keras import initializers, Model, optimizers, callbacks
from keras.models import load_model
#from keras.utils.training_utils import multi_gpu_model
from glob import glob
from keras import optimizers

In [2]:
# Get Paths
from glob import glob
# EEG package
from mne import  pick_types
from mne.io import read_raw_edf
import mne
import os
import numpy as np
# Save the model
import pickle
import tensorflow as tf
from tensorflow import keras
import tensorboard

In [3]:
FNAMES = os.listdir()
for file in FNAMES:
    if ".edf" not in file:
        FNAMES.remove(file)

In [4]:
def get_data(subj_num=FNAMES, epoch_sec=0.0625):
    """ Import from edf files data and targets in the shape of 3D tensor
    
        Output shape: (Trial*Channel*TimeFrames)
        
        Some edf+ files recorded at low sampling rate, 128Hz, are excluded. 
        Majority was sampled at 160Hz.
        
        epoch_sec: time interval for one segment of mashes
        """

    # To calculated completion rate
    count = 0
    
    # Initiate X, y
    X = []
    y = []
    
    # fixed numbers
    nChan = 61
    sfreq = 250
    sliding = epoch_sec/2 
    
    # Sub-function to assign X and X, y
    def append_X(n_segments, old_x):
        '''This function generate a tensor for X and append it to the existing X'''
        new_x = old_x + [data[:, int(sfreq*sliding*n):int(sfreq*sliding*(n+2))] for n in range(n_segments)\
                     if data[:, int(sfreq*sliding*n):int(sfreq*sliding*(n+2))].shape==(nChan, int(sfreq*epoch_sec))]
        return new_x
    print(subj_num)
    for i, subj in enumerate(subj_num):
        # Return completion rate
        count+=1
        displayStep = max(len(subj_num)//10, 1)

        if i%displayStep == 0:
            print('working on {}, {:.1%} completed'.format(subj, count/len(subj_num)))

        # Get file names
        #fnames = os.listdir()
        #for file in fnames:
        #    if ".edf" not in file:
        #        fnames.remove(file)
        #if ".ipynb_checkpoints" in fnames:
        #    fnames.remove(".ipynb_checkpoints")
        #for i, fname in enumerate(fnames):
            
            # Import data into MNE raw object
            fname = subj
            print(fname)
            raw = read_raw_edf(subj, preload=True, verbose=False)
            try:
                ch_names = raw.ch_names
            except:
                continue
            #channels = raw.ch_names
            #for i in channels:
            #    if i != "Fc3." and i != "Fcz." and i != "Fc4." and i != "C3.." and i != "Cz.." and i != "C4.." and i != "Cp3." and i != "Cpz." and i != "Cp4.":
            #        raw.drop_channels(i)
            #raw.filter(7.5,12)

            picks = pick_types(raw.info, eeg=True)
            
            if raw.info['sfreq'] != 250:
                print('{} is sampled at 128Hz so will be excluded.'.format(subj))
                break
            
            # High-pass filtering
            raw.filter(l_freq=1, h_freq=None, picks=picks)
            
            # Get annotation
            try:
                #### for  mne .18
                # events = raw.find_edf_events()
                #### for  mne .19
                events = mne.find_events(raw, verbose = False)
            except:
                continue
            # Get data
            data = raw.get_data(picks=picks)
            
            """ Assignment Starts """ 
            if "Difficult" in fname:

                # Number of sliding windows
                n_segments = int((raw.n_times/(epoch_sec*sfreq))*2-1)
                
                y.extend([2]*n_segments)
                X = append_X(n_segments, X)
                      
            elif "Med" in fname:
                
                # Number of sliding windows
                n_segments = int((raw.n_times/(epoch_sec*sfreq))*2-1)
                
                y.extend([1]*n_segments)
                X = append_X(n_segments, X)
                        
            elif "Easy" in fname:
                   
                # Number of sliding windows
                n_segments = int((raw.n_times/(epoch_sec*sfreq))*2-1)
                
                y.extend([0]*n_segments)
                X = append_X(n_segments, X)
    
    X = np.stack(X)
    y = np.array(y).reshape((-1,1))
    return X, y, ch_names

In [5]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#%%
def convert_mesh(X):
    
    mesh = np.zeros((X.shape[0], X.shape[2], 9, 11))
    X = np.swapaxes(X, 1, 2)
    
    # 1st line
    mesh[:, :, 0, 4:7] = X[:,:,21:24]; print('1st finished')
    
    # 2nd line
    mesh[:, :, 1, 3:8] = X[:,:,24:29]; print('2nd finished')
    
    # 3rd line
    mesh[:, :, 2, 1:10] = X[:,:,29:38]; print('3rd finished')
    
    # 4th line
    mesh[:, :, 3, 1:10] = np.concatenate((X[:,:,38].reshape(-1, X.shape[1], 1),\
                                          X[:,:,0:7], X[:,:,39].reshape(-1, X.shape[1], 1)), axis=2)
    print('4th finished')
    
    # 5th line
    mesh[:, :, 4, 0:11] = np.concatenate((X[:,:,(42, 40)],\
                                        X[:,:,7:14], X[:,:,(41, 43)]), axis=2)
    print('5th finished')
    
    # 6th line
    mesh[:, :, 5, 1:10] = np.concatenate((X[:,:,44].reshape(-1, X.shape[1], 1),\
                                        X[:,:,14:21], X[:,:,45].reshape(-1, X.shape[1], 1)), axis=2)
    print('6th finished')
               
    # 7th line
    mesh[:, :, 6, 1:10] = X[:,:,46:55]; print('7th finished')
    
    # 8th line
    mesh[:, :, 7, 3:8] = X[:,:,55:60]; print('8th finished')
    
    # 9th line
    #mesh[:, :, 8, 4:7] = X[:,:,60]; print('9th finished')
    
    # 10th line
    #mesh[:, :, 9, 5] = X[:,:,63]; print('10th finished')
    
    return mesh

#%%
def prepare_data(X, y, test_ratio=0.2, return_mesh=True, set_seed=42):
    
    # y encoding
    print("y encoding")
    oh = OneHotEncoder()
    y = oh.fit_transform(y).toarray()
    print("Finished y")
    
    # Shuffle trials
    np.random.seed(set_seed)
    trials = X.shape[0]
    shuffle_indices = np.random.permutation(trials)
    X = X[shuffle_indices]
    y = y[shuffle_indices]
    
    # Test set seperation
    train_size = int(trials*(1-test_ratio)) 
    X_train, X_test, y_train, y_test = X[:train_size,:,:], X[train_size:,:,:],\
                                    y[:train_size,:], y[train_size:,:]
    print("Finish train and test data")                                
    # Z-score Normalization
    def scale_data(X):
        shape = X.shape
        scaler = StandardScaler()
        scaled_X = np.zeros((shape[0], shape[1], shape[2]))
        displayStep = max(int(shape[0]/10), 1)
        for i in range(shape[0]):
            for z in range(shape[2]):
                scaled_X[i, :, z] = np.squeeze(scaler.fit_transform(X[i, :, z].reshape(-1, 1)))
            if i%displayStep == 0:
                print('{:.1%} done'.format((i+1)/shape[0]))   
        return scaled_X
            
    X_train, X_test  = scale_data(X_train), scale_data(X_test)
    print("Finished scale data")
    if return_mesh:
        X_train, X_test = convert_mesh(X_train), convert_mesh(X_test)
    print("Finished convert mesh")
    return X_train, y_train, X_test, y_test

In [6]:
X,y,ch_names = get_data(FNAMES, epoch_sec = 0.625)
X_train, y_train, X_test, y_test = prepare_data(X, y)

['P15S1Easy.edf', 'P15S2Easy.edf', 'P08S1Difficult.edf', 'P13S2Medium.edf', 'eeg_preprocessing2.py', '.ipynb_checkpoints', 'Untitled-Copy1.ipynb', 'P01S2DifficultData.edf', 'P03S1Easy.edf', 'P01S1MedData.edf', 'P10S1Difficult.edf', 'P06S2Easy.edf', 'P01S2MediumData.edf', 'P03S1Medium.edf', 'P03S2Difficult.edf', 'P03S2Easy.edf', 'P03S2Medium.edf', 'P04S1Difficult.edf', 'P06S1Easy.edf', 'P06S2Medium.edf', 'P07S2Difficult.edf', 'P07S2Medium.edf', 'P08S1Easy.edf', 'P09S2Medium.edf', 'P10S1Medium.edf', 'P10S2Easy.edf', 'P11S2Easy.edf', 'P13S2Difficult.edf', 'P12S2Easy.edf', 'P14S1Medium.edf', 'P14S2Easy.edf', 'P01S1DifficultData.edf', 'P15S1Medium.edf', 'P12S2Difficulty.edf', 'P01S2EasyData.edf', 'P10S2Medium.edf', 'P02S2Difficult.edf', 'P13S1Difficult.edf', 'P07S1Easy.edf', 'P10S2Difficult.edf', 'P12S1Easy.edf', 'P04S1Medium.edf', 'P12S2Medium.edf', 'P02S1Difficult.edf', 'P02S1Easy.edf', 'P02S2Easy.edf', 'P02S1Medium.edf', 'P03S1Difficult.edf', 'P02S2Medium.edf', 'P04S2Difficult.edf', 'P04

In [7]:
# Make another dimension, 1, to apply CNN for each time frame.
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3], 1)
X_test = X_test.reshape(X_test.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3], 1)

In [8]:
## Complicated Model - the same as Zhang`s
input_shape = X_train.shape[1:5]
lecun = initializers.lecun_normal(seed=42)

In [9]:
# Input layer
inputs = Input(shape=input_shape)
x = inputs
# TimeDistributed Wrapper
def timeDist(layer, prev_layer, name):
    return TimeDistributed(layer, name=name)(prev_layer)  

In [ ]:
x = timeDist(Conv2D(32, (3,3), padding='same', 
                    data_format='channels_last', kernel_initializer=lecun), inputs, name='CNN1')
x = BatchNormalization(name='batch1')(x)
x = Activation('elu', name='act1')(x)
x = timeDist(Conv2D(64, (3,3), padding='same', data_format='channels_last', kernel_initializer=lecun), x, name='CNN2')
x = BatchNormalization(name='batch2')(x)
x = Activation('elu', name='act2')(x)
x = timeDist(Conv2D(128, (3,3), padding='same', data_format='channels_last', kernel_initializer=lecun), x, name='CNN3')
x = BatchNormalization(name='batch3')(x)
x = Activation('elu', name='act3')(x)
x = timeDist(Flatten(), x, name='flatten')

# Fully connected layer block
y = Dense(1024, kernel_initializer=lecun, name='FC')(x)
y = Dropout(0.5, name='dropout1')(y)
y = BatchNormalization(name='batch4')(y)
y = Activation(activation='elu')(y)

# Recurrent layers block
z = LSTM(64, kernel_initializer=lecun, return_sequences=True, name='LSTM1')(y)
z = LSTM(64, kernel_initializer=lecun, name='LSTM2')(z)

# Fully connected layer block
h = Dense(1024, kernel_initializer=lecun, activation='elu', name='FC2')(z)
h = Dropout(0.5, name='dropout2')(h)

# Output layer
outputs = Dense(3, activation='softmax')(h)

# Model compile
model = Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
modelname = "model_all_electrodes_fully_connected"
# Get past models
MODEL_LIST = glob('model/'+modelname+'*')
print(MODEL_LIST)
if MODEL_LIST:
    print('A model that already exists detected and loaded.')
    model = load_model(MODEL_LIST[-1])

In [ ]:
callbacks_list = [callbacks.ModelCheckpoint('./model/'+modelname+'.h5', 
                                            save_best_only=True, 
                                            monitor='val_loss'),
                 callbacks.EarlyStopping(monitor='acc', patience=20),
                 callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50),
                 callbacks.TensorBoard(log_dir='./tensorboard_dir/'+modelname, 
                                       histogram_freq=0, 
                                       write_graph=True,
                                       write_images=True)]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.001), metrics=['acc'])
hist = model.fit(X_train, y_train, batch_size=64, epochs=1000, 
                callbacks=callbacks_list, validation_data=(X_test, y_test))